# Graph Visualization

## requirements.txt


In [ ]:
!pip install -q minio
!pip install -q kglab
!pip install -q sparqlwrapper
!pip install -q pygraphml

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
import kglab
from minio import Minio
from rdflib import Graph, plugin
import plotly.express as px
import pandas as pd
from urllib.request import urlopen
import os,json

from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML 
import networkx as nx
from ipysigma import Sigma

In [ ]:
## Remote SPARQL server
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)



In [5]:
rp1 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?type ?target ?sType ?tType
WHERE {
  graph ?g {
    ?source a ?sType .
    ?target a ?tType .
    ?source ?type ?target .
    FILTER((?sType) IN (schema:Person, schema:Organization, schema:Dataset, schema:Course, schema:Document))
    FILTER((?tType) IN (schema:Person, schema:Organization, schema:Dataset, schema:Course, schema:Document))
  }
}

"""

ep = "http://0.0.0.0:7878/query"

df = get_sparql_dataframe(ep, rp1)
df.head(10)
g = GraphML()

NameError: name 'SPARQLWrapper' is not defined

In [ ]:
#  yeah, I get it...   don't iterate rows...  PR's welcome for this, being bad is too easy!  
# for index, row in nodes.iterrows():
#    g.add_node(row['Id'])

for index, row in df.iterrows():
    n1 = g.add_node(row['source'])
    n1['type'] = row['sType']
    n2 = g.add_node(row['target'])
    n2['type'] = row['tType']
    e = g.add_edge(n1, n2)
    e['predicate'] = row['type']

fname = "./output/testGraphML.xml"
parser = GraphMLParser()
parser.write(g, fname)

In [9]:
# g = nx.les_miserables_graph()
g = nx.read_graphml("./output/exampleGraphML.graphml")

In [8]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=3
)



Sigma(nx.Graph with 5,323 nodes and 8,211 edges)